## LSTM Example

We want to train an univariate, one feature LSTM

**0) Loading Libraries and Subroutines**

Standard libraries for plotting and numerical operations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Loading LSTM related keras libraries:

In [ ]:
from keras import optimizers
from keras.layers import LSTM
from keras.layers import Dense
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler

Calling a subroutine that puts data set in the correct shape for LSTM (see later)

In [ ]:
from prepare_data import prepare_data

<br>

**1) Generating a Simple Dataset**

In [ ]:
t_start = -50
t_end   = 10
incr    = 0.25

t       = np.arange(t_start, t_end, incr)
t       = t.reshape(len(t), 1)
Y_t     = np.sin(t) + 0.1*np.random.randn(len(t), 1) + np.exp((t + 20)*0.05)

In [ ]:
plt.plot(t, Y_t)
plt.title('complete series')
plt.show()

**2) Preprocessing the Data**

2a) Scaling:

In [ ]:
scaler  = MinMaxScaler(feature_range = (0, 1))
Y_tnorm = scaler.fit_transform(Y_t)

2b) Reshaping the Data

The dataset has only one feature. We want to predict *dt_futu* data points into the future and train the LSTM with the previous *dt_past* time steps to predict the next. For that purpose, we need to reshape the data in such a way that *dt_futu* is interpreted as features. Therefore, we run the following subroutine:<br>
<br>

In [ ]:
dt_past    = 20
dt_futu    = 8
n_features = 1

[X, Y] = prepare_data(Y_tnorm, dt_past, dt_futu)

<br>

Let us check the shapes:

In [ ]:
print(Y_tnorm.shape)
print(X.shape)
print(Y.shape)

In order to train the LSTM with $dt_{futu}$ and $dt_{past}$, we need the first $dt_{past}$ steps in order to train for the next prediction. The lenght of the period we want to predict is $dt_{futu}$. The last step in the training will be the point, when we predict the last sequence $dt_{futu}$ with the previous steps $dt_{past}$. Thus, the training data set has the length $len\left(Y_{tnorm}\right) - dt_{past} - dt_{futu} + 1$. Each time point in the past we need for the prediction will have its own influence on the future and therefore are regarded as feature. Thus, $X$ has to have the shape $\left[len\left(Y_{tnorm}\right) - dt_{past} - dt_{futu} + 1\right] \times dt_{past}$.<br>   
Because of that interpretation, each feature is shifted by one time point:

In [ ]:
print(X[:dt_past, :dt_past, 0])

<br>

Following the same logic, in order to train the LSTM to predict the future $dt_{futu}$ ahead, $Y$ needs to have the same structure: 

In [ ]:
print(Y[:dt_futu, :dt_futu, 0])

<br>

2c) Splitting data into Training and Test dataset

In [ ]:
cut            = int(np.round(0.7*Y_tnorm.shape[0]))

TrainX, TrainY = X[:cut], Y[:cut]
TestX,   TestY = X[cut:], Y[cut:]

Just to make sure, that the shape is *len(X) x dt_past x n_features*

In [ ]:
print(TrainX.shape)

<br>

**3) Creating the Model**

Creating a LSTM in Python follows the same syntax as for CNNs. We want to predict data points in the future, based on previous data points. Therefore, we need to solve a **regression** problem:<br>

In [ ]:
n_neurons  = 400
batch_size = 128

model = Sequential()
model.add(LSTM(n_neurons, input_shape = (dt_past, n_features), activation = 'tanh'))
model.add(Dense(dt_futu))

opt = optimizers.Adam()
model.compile(loss = 'mean_squared_error', optimizer = opt)

model.summary()

<br>

**4) Running the Fit**

In [ ]:
n_epochs = 800
out = model.fit(TrainX, TrainY, epochs = n_epochs, batch_size = batch_size, validation_split = 0.2, verbose = 2, shuffle = False)

In [ ]:
#plotting #############################################################
plt.plot(out.history['loss'])
plt.plot(out.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc = 'upper left')
plt.savefig('training loss.pdf')
plt.show()
#######################################################################

<br>

**5) Evaluating the Fit**

In [ ]:
PredY = model.predict(TestX)
back  = PredY.shape[0]

plt.plot(t, Y_tnorm, linewidth = 5)
plt.plot(t[-back:], PredY[:, dt_futu-1])
plt.legend(['actual data', 'prediction'])
plt.fill_between([t[-back,0], t[-1,0]], 0, 1, color = 'k', alpha = 0.1)
plt.plot([t[-back,0], t[-back,0]], [0, 1], 'k-', linewidth = 3)
plt.show()

In [ ]:
print(PredY.shape)

In [ ]:
print(TestX.shape)